In [1]:
#conda activate bertopic
import pandas as pd
colnames=(['id', 'text', 'emotion', 'intensity'])
anger_train = pd.read_csv('/home/chris/Downloads/tweet-ids-001/SarahEverard/WASSA17_EmoInt/anger_train.txt',sep='\t', header=None, names=colnames)
anger_train2 = pd.read_csv('/home/chris/Downloads/tweet-ids-001/SarahEverard/WASSA17_EmoInt/anger_train2.txt',sep='\t', header=None, names=colnames)
anger_train3 = pd.read_csv('/home/chris/Downloads/tweet-ids-001/SarahEverard/WASSA17_EmoInt/anger_train3.txt',sep='\t', header=None, names=colnames)
joy_train = pd.read_csv('/home/chris/Downloads/tweet-ids-001/SarahEverard/WASSA17_EmoInt/joy_train.txt',sep='\t', header=None, names=colnames)
joy_train2 = pd.read_csv('/home/chris/Downloads/tweet-ids-001/SarahEverard/WASSA17_EmoInt/joy_train2.txt',sep='\t', header=None, names=colnames)
joy_train3 = pd.read_csv('/home/chris/Downloads/tweet-ids-001/SarahEverard/WASSA17_EmoInt/joy_train3.txt',sep='\t', header=None, names=colnames)
fear_train = pd.read_csv('/home/chris/Downloads/tweet-ids-001/SarahEverard/WASSA17_EmoInt/fear_train.txt',sep='\t', header=None, names=colnames)
fear_train2 = pd.read_csv('/home/chris/Downloads/tweet-ids-001/SarahEverard/WASSA17_EmoInt/fear_train2.txt',sep='\t', header=None, names=colnames)
fear_train3 = pd.read_csv('/home/chris/Downloads/tweet-ids-001/SarahEverard/WASSA17_EmoInt/fear_train3.txt',sep='\t', header=None, names=colnames)
sadness_train = pd.read_csv('/home/chris/Downloads/tweet-ids-001/SarahEverard/WASSA17_EmoInt/sadness_train.txt',sep='\t', header=None, names=colnames)
sadness_train2 = pd.read_csv('/home/chris/Downloads/tweet-ids-001/SarahEverard/WASSA17_EmoInt/sadness_train2.txt',sep='\t', header=None, names=colnames)
sadness_train3 = pd.read_csv('/home/chris/Downloads/tweet-ids-001/SarahEverard/WASSA17_EmoInt/sadness_train3.txt',sep='\t', header=None, names=colnames)

emotions = pd.concat([anger_train, anger_train2, anger_train3, joy_train, joy_train2, joy_train3, fear_train, fear_train2, fear_train3, sadness_train, sadness_train2, sadness_train3])
print(emotions.shape)
#(7102, 4)

(7102, 4)


In [5]:
count = emotions['text'].str.split().apply(len).value_counts()
count

22    391
19    369
18    360
21    352
23    347
17    343
20    341
16    324
24    321
8     316
11    315
14    308
15    303
10    301
12    300
9     274
13    271
25    259
6     222
7     217
26    190
5     159
27    129
4     122
3      76
28     70
29     51
2      35
30     20
31      9
1       4
32      2
33      1
Name: text, dtype: int64

In [2]:
##Remove @mentions:
import re
emotions['text_processed'] = emotions['text'].apply(lambda x: re.sub("@[A-Za-z0-9_]+","",x))

In [3]:
##Apply undersampling to reduce all datasets to 1300 (leaving enough for test dataset too)
sadness = emotions[emotions['emotion']=='sadness'].sample(n=1300, random_state=1)
anger = emotions[emotions['emotion']=='anger'].sample(n=1300, random_state=1)
joy = emotions[emotions['emotion']=='joy'].sample(n=1300, random_state=1)
fear = emotions[emotions['emotion']=='fear'].sample(n=1300, random_state=1)

emotions_train = pd.concat([sadness,anger, joy, fear])
emotions_train

emotions_train_ids = emotions_train.id.unique()
emotions_test = emotions[~emotions.id.isin(emotions_train_ids)]
#Check for overlap - should be zero
pd.concat([emotions_train, emotions_test]).id.duplicated().sum()

0

In [4]:
#https://towardsdatascience.com/multi-class-text-classification-with-deep-learning-using-bert-b59ca2f5c613
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

In [5]:
possible_labels = emotions.emotion.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
print(label_dict)
#Replace the str value in 'emotion' with the corresponding label in label_dict
emotions['label'] = emotions.emotion.replace(label_dict)

{'anger': 0, 'joy': 1, 'fear': 2, 'sadness': 3}


In [23]:
##Remove non-ASCII characters
emotions['text_processed_ASCII_only'] = emotions['text_processed'].str.encode('ascii', 'ignore').str.decode('ascii')
emotions[emotions.text_processed_ASCII_only.str.contains('http')]
#No obvious URLs present

,id,text,emotion,intensity,text_processed,label,text_processed_ASCII_only


In [7]:
from sklearn.model_selection import train_test_split
df = emotions.copy()
df = df.reset_index(drop=True)
df = df.drop('intensity', 1)
df = df.drop('text', 1)
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

len(X_train)
#6036
len(X_val)
#1066
len(y_train)
#6036
len(y_val)
#1066

#Add str 'not_set' to a column of list df.shape[0] (i.e same as len(df))
df['data_type'] = ['not_set']*df.shape[0]

#Set 'data_type' column to 'train' or 'val' depending on the index numbers in the X_train and X_val datasets
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['emotion', 'label', 'data_type']).count()

/home/chris/anaconda3/envs/bertopic/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.
/home/chris/anaconda3/envs/bertopic/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """


id  text_processed  text_processed_ASCII_only
emotion label data_type                                                 
anger   0     train      1446            1446                       1446
              val         255             255                        255
fear    2     train      1914            1914                       1914
              val         338             338                        338
joy     1     train      1373            1373                       1373
              val         243             243                        243
sadness 3     train      1303            1303                       1303
              val         230             230                        230

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', 
                                          do_lower_case=False)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text_processed_ASCII_only.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt' 
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text_processed_ASCII_only.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

##Create dataset which is subsequently loaded into dataloaders
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/chris/anaconda3/envs/bertopic/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


## BERT fine-tuning (skip to classification):

In [6]:
model = BertForSequenceClassification.from_pretrained("bert-base-cased", 
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)
model.cuda()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [11]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 8

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [14]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=2e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)


In [13]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten() #np.argmax returns the indices of the maximum values along an axis.
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted') ##Compares the ground-truth labels (1-D array) to predictions 1-D array to calculate F1 score; Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label)

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat): #hence print accuracy for each multi-class label
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    #torch.save(model.state_dict(), f'/home/chris/Downloads/tweet-ids-001/SarahEverard/WASSA17_EmoInt/Models/{epoch}/')
    model.save_pretrained(f'/home/chris/Downloads/tweet-ids-001/SarahEverard/WASSA17_EmoInt/Models/Chiorrini_paper/cased_ASCIIonly/{epoch}/') 
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

In [ ]:
'''
Results:

----------------------------------
Try replicating this paper: http://ceur-ws.org/Vol-2841/DARLI-AP_17.pdf
learning_rate 2e-5
train_batch_size 8
eval_batch_size 8
max_seq_length 95
adam_epsilon 1e-8

Found that bert-base-cased gave slightly better results; 2 epochs was the best
The uncased BERT has accuracy = 0.89 and 𝐹1 = 0.89, while the cased version has accuracy = 0.90 and 𝐹1=0.91:


My results:
Epoch 1
Training loss: 1.1682064250259605
Validation loss: 0.6271203979293802
F1 Score (Weighted): 0.7695232866315141

Epoch 2
Training loss: 0.5172508799680663
Validation loss: 0.5258828048635997
F1 Score (Weighted): 0.8224203716636478

##Hence use Epoch 2 as from here validation loss starts to increase

Epoch 3
Training loss: 0.3489388401061913
Validation loss: 0.571927275972566
F1 Score (Weighted): 0.8232465252939838

Epoch 4
Training loss: 0.25141982606269836
Validation loss: 0.7232665289297644
F1 Score (Weighted): 0.8297407987291848

Epoch 5
Training loss: 0.17712804765713444
Validation loss: 0.8141401502900625
F1 Score (Weighted): 0.8318911161429177

Epoch 6
Training loss: 0.12685735518350236
Validation loss: 0.8371931063258888
F1 Score (Weighted): 0.8333345175059841
-----------------------------------------------
Finally, try remove rows with non-ASCII characters only and re-classify using same parameters as paper:

bert-base-cased
learning_rate 2e-5
train_batch_size 8
eval_batch_size 8
max_seq_length 95
adam_epsilon 1e-8

Epoch 1
Training loss: 0.6469730544267901
Validation loss: 0.4050789241515684
F1 Score (Weighted): 0.8688711474083493

Epoch 2
Training loss: 0.3007535682772194
Validation loss: 0.42748885782463336
F1 Score (Weighted): 0.8706080773616959

##Hence use Epoch 2

Epoch 3
Training loss: 0.18287871097020236
Validation loss: 0.613139122538616
F1 Score (Weighted): 0.8639991431502831

Epoch 4
Training loss: 0.13386496885930996
Validation loss: 0.6764722590412034
F1 Score (Weighted): 0.8512102017117004
----------------------------------------------------------
Decision to use epoch 2, BERT-cased model with non-ASCII characters removed

'''

## Pick up on classification here (skipped to classification from above)

In [7]:
##Now re-load fine-tuned model (use cased_ASCIIonly, epoch 2 as seems to be best):

import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

import random
import numpy as np

tokenizer = BertTokenizer.from_pretrained('bert-base-cased', 
                                          do_lower_case=False)
model = BertForSequenceClassification.from_pretrained("/home/chris/Downloads/tweet-ids-001/SarahEverard/WASSA17_EmoInt/Models/Chiorrini_paper/cased_ASCIIonly/2/",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

In [9]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat): #hence print accuracy for each multi-class label
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')
        

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)




In [18]:
batch_size = 4

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

  0%|          | 0/267 [00:00<?, ?it/s]

Class: anger
Accuracy: 215/255

Class: joy
Accuracy: 221/243

Class: fear
Accuracy: 285/338

Class: sadness
Accuracy: 207/230



In [56]:
preds = np.argmax(predictions, axis=1).flatten()
data = [preds.tolist(), true_vals.tolist()]
results = pd.DataFrame(data).transpose()
results_concat = pd.concat([results.reset_index(), df[df.data_type=='val'].reset_index()
], axis=1)
results_concat

,index,0,1,index,id,emotion,text_processed,label,text_processed_ASCII_only,data_type
0,0,0,0,1,10001,anger,So my Indian Uber driver just called someone t...,0,So my Indian Uber driver just called someone t...,val
1,1,0,0,13,10013,anger,Why does have to come to Glasgow on a night I...,0,Why does have to come to Glasgow on a night I...,val
2,2,0,0,14,10014,anger,Fuking fuming 😤,0,Fuking fuming,val
3,3,0,0,20,10020,anger,Making my buddy cry !! Bitch wait for revenge 🤗👌🏻,0,Making my buddy cry !! Bitch wait for revenge,val
4,4,0,0,21,10021,anger,Bes! You don't just tell a true blooded hoopju...,0,Bes! You don't just tell a true blooded hoopju...,val
...,...,...,...,...,...,...,...,...,...,...
1061,1061,3,3,7071,41502,sadness,Probs spent a grand total of five minutes sobe...,3,Probs spent a grand total of five minutes sobe...,val
1062,1062,2,3,7080,41511,sadness,yeah agree - I think it was a family member a...,3,yeah agree - I think it was a family member a...,val
1063,1063,1,3,7084,41515,sadness,I saw those dreams dashed &amp;&amp; divided l...,3,I saw those dreams dashed &amp;&amp; divided l...,val
1064,1064,3,3,7096,41527,sadness,"She used to be beautiful, but she lived her li...",3,"She used to be beautiful, but she lived her li...",val


## Now work to apply to SE datset:

In [10]:
import pandas as pd
import csv
import ast
from tqdm import tqdm
tqdm.pandas()
dtypes_demog_VADER_NRC={
        'created_at'  :  str, 'id'  :  str, 'full_text'  :  str, 'RT_full_text'  :  str, 'source'  :  str, 'is_quote_status'  :  str,
       'retweeted_status'  :  str, 'retweet_count'  :  float, 'favorite_count'  :  float, 'user_id_str'  :  str,
       'screen_name'  :  str, 'name'  :  str, 'description'  :  str, 'hashtags_all'  :  str, 'gender_male'  :  float,
       'gender_female'  :  float, 'age_<=18'  :  float, 'age_19-29'  :  float, 'age_30-39'  :  float, 'age_>=40'  :  float,
       'org_non-org'  :  float, 'org_is-org'  :  float, 'Gender'  :  str, 'Org?'  :  str, 'Age'  :  str,
       'text_banned' : str, 'full_text_VADER' : str, 'text_neg' : float, 'text_neu' : float, 'text_pos' : float,
       'text_compound' : float, 'text_banned_neg' : float, 'text_banned_neu' : float, 'text_banned_pos' : float,
       'text_banned_compound' : float, 'full_text_all' : str, 'RT_full_text' : str, 'RT_full_text_VADER' : str,
       'emotion': str, 'Major_emotion': str, 'Major_emotion_emot': str,
       'anger_rel_emot': float, 'anticipation_rel_emot': float, 'disgust_rel_emot': float, 'fear_rel_emot': float, 
       'joy_rel_emot': float, 'sadness_rel_emot': float, 'surprise_rel_emot': float, 'trust_rel_emot': float
       }
df_harass_selected_demog_VADER_NRC_chunk = pd.read_csv('~/Downloads/tweet-ids-001/SarahEverard/df_harass_selected_demog_VADER_NRC.csv',
     sep=',', 
     quotechar='"', 
     chunksize=100000,
     dtype=dtypes_demog_VADER_NRC,
     index_col=0)
df_harass_selected_demog_VADER_NRC = pd.concat(df_harass_selected_demog_VADER_NRC_chunk)
df_harass_selected_demog_VADER_NRC.shape
#(1039948, 48)

(1039948, 48)

In [12]:
df_harass_selected_demog_VADER_NRC = df_harass_selected_demog_VADER_NRC[['created_at', 'id', 'full_text_all']]

In [ ]:
##Remove @mentions:
import re
df_harass_selected_demog_VADER_NRC['full_text_all_processed'] = df_harass_selected_demog_VADER_NRC['full_text_all'].apply(lambda x: re.sub("@[A-Za-z0-9_]+","",x))
df_harass_selected_demog_VADER_NRC['full_text_all_processed'] = df_harass_selected_demog_VADER_NRC['full_text_all_processed'].str.encode('ascii', 'ignore').str.decode('ascii')
df_harass_selected_demog_VADER_NRC['full_text_all_processed']  = df_harass_selected_demog_VADER_NRC['full_text_all_processed'] .str.replace(r'https?://[^\s<>"]+|www\.[^\s<>"]+', " ")

In [14]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

import random
import numpy as np

tokenizer = BertTokenizer.from_pretrained('bert-base-cased', 
                                          do_lower_case=False)
model = BertForSequenceClassification.from_pretrained("/home/chris/Downloads/tweet-ids-001/SarahEverard/WASSA17_EmoInt/Models/Chiorrini_paper/cased_ASCIIonly/2/",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat): #hence print accuracy for each multi-class label
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')
        

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', 
                                          do_lower_case=False)

encoded_data_val = tokenizer.batch_encode_plus(
    df_harass_selected_demog_VADER_NRC.full_text_all_processed.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
#labels_val = torch.tensor(df[df.data_type=='val'].label.values)
#Create false labels
labels_val = torch.tensor([0]*len(df_harass_selected_demog_VADER_NRC))

dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/chris/anaconda3/envs/bertopic/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [44]:
batch_size = 180

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [45]:
import gc
#del variables
gc.collect()

146

In [ ]:
import torch
torch.cuda.empty_cache()

_2, predictions2, true_vals2 = evaluate(dataloader_validation)

In [48]:
preds = np.argmax(predictions2, axis=1).flatten()
data = [preds.tolist(), true_vals2.tolist()]
results = pd.DataFrame(data).transpose()

In [65]:
results_concat = pd.concat([results.reset_index(), df_harass_selected_demog_VADER_NRC.reset_index()], axis=1)
results_concat = results_concat.rename(columns={0 : 'emotion_label'})
results_concat = results_concat.drop(columns=[1, 'index'])
results_concat.shape
#(1039948, 5)
results_concat.columns
#Index([ 'emotion_label', 'created_at', 'id', 'full_text_all',
#       'full_text_all_processed'],
#      dtype='object')

#results_concat.to_csv('~/Downloads/tweet-ids-001/SarahEverard/df_harass_selected_demog_VADER_NRC_BERTemotion.csv', sep=',', quotechar='"', quoting=csv.QUOTE_ALL, chunksize=100000)


Index(['emotion_label', 'created_at', 'id', 'full_text_all',
       'full_text_all_processed'],
      dtype='object')

In [ ]:
##See Harassment_final_code.ipynb for emotion_analysis